In [2]:
import csv
import sqlite3

# Define the path to the CSV file
csv_file_path = 'data_realestate.csv'

# Read the CSV file
with open(csv_file_path, mode='r') as file:
    csv_reader = csv.reader(file)
    headers = next(csv_reader)
    rows = list(csv_reader)

# Print the headers and first few rows to understand the structure
print("Headers:", headers)
print("First few rows:", rows[:5])


Headers: ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'street', 'city', 'statezip']
First few rows: [['313000', '3', '1.5', '1340', '7912', '1.5', '0', '0', '3', '1340', '0', '1955', '2005', '18810 Densmore Ave N', 'Shoreline', 'WA 98133'], ['2384000', '5', '2.5', '3650', '9050', '2', '0', '4', '5', '3370', '280', '1921', '0', '709 W Blaine St', 'Seattle', 'WA 98119'], ['342000', '3', '2', '1930', '11947', '1', '0', '0', '4', '1930', '0', '1966', '0', '26206-26214 143rd Ave SE', 'Kent', 'WA 98042'], ['420000', '3', '2.25', '2000', '8030', '1', '0', '0', '4', '1000', '1000', '1963', '0', '857 170th Pl NE', 'Bellevue', 'WA 98008'], ['550000', '4', '2.5', '1940', '10500', '1', '0', '0', '4', '1140', '800', '1976', '1992', '9105 170th Ave NE', 'Redmond', 'WA 98052']]


In [3]:
import sqlite3
import csv

# Define the path to the CSV file
csv_file_path = 'data_realestate.csv'

# Function to create database connection
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return conn

# Function to create tables
def create_tables(conn):
    try:
        cursor = conn.cursor()
        
        # Create PropertyType table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS PropertyType (
                id INTEGER PRIMARY KEY,
                type_name TEXT NOT NULL UNIQUE
            )
        ''')
        
        # Create Location table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Location (
                id INTEGER PRIMARY KEY,
                location_name TEXT NOT NULL UNIQUE
            )
        ''')
        
        # Create Property table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Property (
                id INTEGER PRIMARY KEY,
                property_type_id INTEGER,
                location_id INTEGER,
                price REAL,
                size REAL,
                FOREIGN KEY (property_type_id) REFERENCES PropertyType (id),
                FOREIGN KEY (location_id) REFERENCES Location (id)
            )
        ''')
        
        conn.commit()
    except sqlite3.Error as e:
        print(e)

# Function to insert data into normalized tables
def insert_data(conn, csv_file_path):
    try:
        cursor = conn.cursor()
        
        # Read the CSV file
        with open(csv_file_path, mode='r') as file:
            csv_reader = csv.reader(file)
            headers = next(csv_reader)
            
            for row in csv_reader:
                property_type = row[1]
                location = row[2]
                price = float(row[3])
                size = float(row[4])
                
                # Insert or ignore into PropertyType table
                cursor.execute('''
                    INSERT OR IGNORE INTO PropertyType (type_name)
                    VALUES (?)
                ''', (property_type,))
                
                # Get PropertyType ID
                cursor.execute('''
                    SELECT id FROM PropertyType WHERE type_name = ?
                ''', (property_type,))
                property_type_id = cursor.fetchone()[0]
                
                # Insert or ignore into Location table
                cursor.execute('''
                    INSERT OR IGNORE INTO Location (location_name)
                    VALUES (?)
                ''', (location,))
                
                # Get Location ID
                cursor.execute('''
                    SELECT id FROM Location WHERE location_name = ?
                ''', (location,))
                location_id = cursor.fetchone()[0]
                
                # Insert into Property table
                cursor.execute('''
                    INSERT INTO Property (property_type_id, location_id, price, size)
                    VALUES (?, ?, ?, ?)
                ''', (property_type_id, location_id, price, size))
        
        conn.commit()
    except sqlite3.Error as e:
        print(e)

# Main function to normalize CSV data and insert into database
def main():
    database = 'realestate_normalized.db'
    
    # Create a database connection
    conn = create_connection(database)
    if conn is not None:
        # Create tables
        create_tables(conn)
        # Insert data
        insert_data(conn, csv_file_path)
    else:
        print("Error! Cannot create the database connection.")
    
    if conn:
        conn.close()

# Run the main function
main()


unable to open database file
Error! Cannot create the database connection.
